In [28]:
# import all the necessary modules
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import OpenAIEmbeddings
import textwrap
import dotenv
import os

In [ ]:
# load OpenAI API key from secret file
dotenv.load_dotenv()
# language model to use
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0) # could also use Ollama

In [29]:
# define a function to print the response in wrapped format
def print_response(response, width=70):
    # wrap the response to fit within the specified width
    wrapper = textwrap.TextWrapper(width=width) 
    # wrap the response
    wrapped_string = wrapper.fill(response)
    # print the wrapped response
    print(wrapped_string)

In [30]:
# set Ollama as model
llm = Ollama(model="llama3")
# test the model with a prompt
llm_response = llm.invoke("Tell me a joke on neuroscientist and language models")
print_response(llm_response)

In [31]:
# create some mock documents to test rag
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [34]:
# NB: with OllamaEmbeddings it doen't work -----
# create a Chroma vesctor store instance with the Ollama embeddings
# vectorstore = Chroma.from_documents(
#     documents,
#     embedding=OllamaEmbeddings(model="llama3"),
# )
# -------> OPEN ISSUE IN OLLAMA EMBEDDINGS <-------

# NB: with OpenAI embeddings it does -----
vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

# to clear the database (needed if running multiple times to avoid accumulation)
# vectorstore.delete_collection() 

In [35]:
# test the vectorstore with a query
vectorstore.similarity_search_with_score(query='simple to care for', k = 6)

Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


[(Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
  0.2768577039241791),
 (Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  0.39694297313690186),
 (Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  0.40311747789382935),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.4430447816848755),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.47815245389938354)]

In [37]:
# define a retriever to search for the most similar (k=1) document
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
# define a message to be filled in by runnable passthrough question to ask a question using the provided context
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
# create a chat prompt template from the message
prompt = ChatPromptTemplate.from_messages([("human", message)])
# define the rag (retrieval augmented generation) chain with a runnable passthrough question
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [38]:
# invoke the rag chain passing through a question
response = rag_chain.invoke("tell how easy is to take care of pets with scales")
# print model response
print_response(response)

Based on the provided context, it seems that taking care of goldfish
with scales is quite easy. The document states that goldfish are
"popular pets for beginners" and require "relatively simple care".
This suggests that caring for goldfish with scales is a
straightforward process that doesn't require specialized knowledge or
skills.
